# Deep Q-Network (DQN) for Atari-Pong
---
In this notebook, we will implement a DQN agent with OpenAI Gym's Pong-v0 environment.
The main challenge is that we train the Deep Reinforcement Learning agent directly using pixels as inputs.

### 1. Import the Necessary Packages

In [1]:
import gym
import random
import torch
import numpy as np
from gym import spaces
import cv2
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

### 2. Instantiate the Environment and Agent

Initialize the environment in the code cell below. The WarpFrame class is a utility class used to translate the original RGB image provided by the OpenAi Gym environment (260, 160, 3) into a grayscale image of dimension (1, 84, 84), as in the original DQN paper.

In [2]:
env = gym.make('Pong-v0')

class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env, width=84, height=84, grayscale=True):
        """Warp frames to 84x84 as done in the Nature paper and later work."""
        gym.ObservationWrapper.__init__(self, env)
        self.width = width
        self.height = height
        self.grayscale = grayscale
        if self.grayscale:
            self.observation_space = spaces.Box(low=0, high=255,
                shape=(self.height, self.width, 1), dtype=np.uint8)
        else:
            self.observation_space = spaces.Box(low=0, high=255,
                shape=(self.height, self.width, 3), dtype=np.uint8)

    def observation(self, frame):
        if self.grayscale:
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (self.width, self.height), interpolation=cv2.INTER_AREA)
        if self.grayscale:
            frame = np.expand_dims(frame, -1)
            
        frame = np.reshape(frame, (1, 84, 84))
        return frame

env = WarpFrame(env)

env.seed(0)
print('State shape: ', env.observation_space.shape)
print('Number of actions: ', env.action_space.n)

C:\Users\SalandMax\Anaconda3\lib\site-packages\gym\envs\registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


State shape:  (84, 84, 1)
Number of actions:  6


Watch an untrained agent playing Pong

In [3]:
from dqn_agent import Agent

agent = Agent(in_channels=1, action_size=env.action_space.n, seed=0)

# watch an untrained agent
state = env.reset()
for j in range(200):
    action = agent.act(state)
    env.render()
    state, reward, done, _ = env.step(action)
    if done:
        break 
        
env.close()

### 3. Train the Agent with DQN

Run the code cell below to train the agent from scratch.  You are welcome to amend the supplied values of the parameters in the function, to try to see if you can get better performance!

Alternatively, you can skip to the next step below (**4. Watch a Smart Agent!**), to load the saved model weights from a pre-trained agent.

In [ ]:
def dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.05, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            next_state, reward, done, _ = env.step(action)
            reward = np.clip(reward, -1, 1)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = 0.05 #max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 10 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=18.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            break
    return scores

scores = dqn()

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 10	Average Score: -17.40
Episode 20	Average Score: -16.95
Episode 30	Average Score: -17.00
Episode 40	Average Score: -16.95
Episode 50	Average Score: -16.98
Episode 60	Average Score: -16.62
Episode 70	Average Score: -16.47
Episode 80	Average Score: -16.38
Episode 90	Average Score: -16.21
Episode 100	Average Score: -15.78
Episode 110	Average Score: -15.27
Episode 120	Average Score: -14.92
Episode 130	Average Score: -14.38
Episode 140	Average Score: -13.91
Episode 150	Average Score: -13.67
Episode 160	Average Score: -13.42
Episode 170	Average Score: -12.89
Episode 180	Average Score: -12.19
Episode 190	Average Score: -11.64
Episode 200	Average Score: -11.36
Episode 210	Average Score: -11.08
Episode 220	Average Score: -10.73
Episode 230	Average Score: -10.37
Episode 240	Average Score: -9.967
Episode 250	Average Score: -9.32
Episode 260	Average Score: -9.01
Episode 270	Average Score: -8.87
Episode 280	Average Score: -8.85
Episode 290	Average Score: -8.62
Episode 300	Average Score: -

### 4. Watch a Smart Agent!

In the next code cell, you will load the trained weights from file to watch a smart agent!

In [ ]:
# load the weights from file
agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))

for i in range(3):
    state = env.reset()
    for j in range(200):
        action = agent.act(state)
        env.render()
        state, reward, done, _ = env.step(action)
        if done:
            break 
            
env.close()

### 5. Explore

In this exercise, you have implemented a DQN agent and demonstrated how to use it to solve an OpenAI Gym environment.  To continue your learning, you are encouraged to complete any (or all!) of the following tasks:
- Amend the various hyperparameters and network architecture to see if you can get your agent to solve the environment faster.  Once you build intuition for the hyperparameters that work well with this environment, try solving a different OpenAI Gym task with discrete actions!
- You may like to implement some improvements such as prioritized experience replay, Double DQN, or Dueling DQN! 
- Write a blog post explaining the intuition behind the DQN algorithm and demonstrating how to use it to solve an RL environment of your choosing.  